- #### Transformaciones
    - ##### sort/orderBy
    - ##### dropDuplicates
    - ##### distinct
    - ##### groupBy
    - ##### union/unionAll
    - ##### unionByName

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("sesion_1") \
        .master("local[*]") \
        .getOrCreate()

In [ ]:

base_path = "../../resources/data/parquet/"
contracts_df = spark.read.parquet(base_path + "contracts_tmp")

contracts_df.show()
contracts_df.printSchema()

In [ ]:
# sort/orderBy

import pyspark.sql.functions as f

contracts_df.orderBy(f.col("cod_iuc").desc(), f.col("fec_alta_fin").desc()).show()

In [ ]:
# dropDuplicates distinct

#contracts_df.dropDuplicates(["cod_iuc", "cod_producto"]).show()

#contracts_df.distinct().show()

In [ ]:
# groupBy -> count, max, min, avg, mean, sum

#contracts_df.groupBy("cod_iuc").count().show()
#contracts_df.groupBy(f.col("cod_iuc")).sum("diff").show()
#contracts_df.groupBy(f.col("cod_iuc")).max("fec_alta_ini").show()

contracts_df.groupBy(f.col("cod_iuc"), f.col("activo"))\
    .agg(
        f.count("*").alias("count"),
        f.sum("diff").alias("sum_diff"),
        f.max("diff").alias("max_diff"),
        f.min("diff").alias("min_diff"),
        f.mean("diff").alias("mean_diff"),
        f.avg("diff").alias("avg_diff"),
        f.max("fec_alta_ini").alias("max_fec_alta_ini")
    ).show()


In [ ]:
contracts_df\
    .groupBy("cod_iuc") \
    .pivot("activo") \
    .agg(
        f.count("*"),
        f.max("fec_alta_fin"),
        f.avg("diff")
    ) \
    .show()

In [ ]:
## union/unionAll unionByName
contracts_tmp_df = contracts_df.filter(f.col("activo").isNull())

contracts_tmp_df.show()

contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")\
    .union(contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")).show()

contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")\
    .unionByName(contracts_tmp_df.select("actual_date","cod_titular","fec_alta_ini")).show()